In [16]:
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F

import trimesh

from src.cleansed_cube import SourceCube, make_cube_faces, Mesh

In [2]:
if False:
    faces = make_cube_faces(n)
    vertices = sphered.permute(0, 2, 3, 1).reshape(-1, 3)
    f = './cube_sphere.stl'
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    mesh.export(f);

In [10]:
def cube_to_sphere(bxyz):
    x, y, z = bxyz[:, 0, :, :], bxyz[:, 1, :, :], bxyz[:, 2, :, :]
    r = torch.sqrt(x**2 + y**2 + z**2)
    x_angle = torch.atan2(y, x)
    z_angle = torch.acos(z/r)
    
    r = torch.ones_like(x)
    x = r * torch.sin(z_angle) * torch.cos(x_angle)
    y = r * torch.sin(z_angle) * torch.sin(x_angle)
    z = r * torch.cos(z_angle)        
    return torch.stack((x, y, z), dim=1)   

def make_cube_sides(n, r=0.5, stacked=False):
    start, end =  -r, +r
    d1, d2 = torch.meshgrid(
        torch.linspace(start, end, steps=n),
        torch.linspace(start, end, steps=n))
    d3 = torch.full_like(d1, end) + 1 / n
    sides = OrderedDict({
        'front': torch.stack((+d3,  d1,  d2), dim=0),
        'right': torch.stack(( d1, +d3,  d2), dim=0),    
        'back' : torch.stack((-d3,  d1,  d2), dim=0),         
        'left' : torch.stack(( d1, -d3,  d2), dim=0),
        'top'  : torch.stack(( d1,  d2, +d3), dim=0),
        'down' : torch.stack(( d1,  d2, -d3), dim=0),
    })
    if stacked:
        return torch.stack([p for p in sides.values()])
    return sides

def to_vertices(t):
    return t.permute(0, 2, 3, 1).reshape(-1, 3)

class SourceCube(nn.Module):
    def __init__(self, n):
        super(SourceCube, self).__init__()        
        stacked = make_sides(n, stacked=True)         
        sphered = cube_to_sphere(stacked)
        vertices = sphered.permute(0, 2, 3, 1).reshape(-1, 3)
        self.register_buffer('vertices', vertices)
        self.register_buffer('faces', make_cube_faces(n).int())
        self.register_buffer('colors', torch.ones_like(vertices) * 0.5)

    def to_homogeneous(_, t):
        return torch.cat((t,
            torch.ones(t.size(0), 1, device=t.device)), dim=-1)

    def forward(self, deform_verts):
        vert = self.vertices + deform_verts
        pos = self.to_homogeneous(vert)[None]
        return Mesh(pos, self.faces, self.colors)

In [6]:
n = 8
faces = make_cube_faces(n)
stacked = make_sides(n, stacked=True)
stacked.shape

torch.Size([6, 3, 8, 8])

In [15]:
def cube_to_ellipsoid(bxyz, rs):    
    x, y, z = bxyz[:, 0, :, :], bxyz[:, 1, :, :], bxyz[:, 2, :, :]
    r = torch.sqrt(x**2 + y**2 + z**2)
    x_angle = torch.atan2(y, x)
    z_angle = torch.acos(z/r)
    
    r = torch.ones_like(x)
    x = rs[0] * torch.sin(z_angle) * torch.cos(x_angle)
    y = rs[1] * torch.sin(z_angle) * torch.sin(x_angle)
    z = rs[2] * torch.cos(z_angle) 
    return torch.stack((x, y, z), dim=1)

def get_xz_angles(bxyz):
    x, y, z = bxyz[:, 0, :, :], bxyz[:, 1, :, :], bxyz[:, 2, :, :]
    r = torch.sqrt(x**2 + y**2 + z**2)
    x_angle = torch.atan2(y, x)
    z_angle = torch.acos(z/r)
    return x_angle, z_angle

def get_ellipsoidal(x_angle, z_angle, rs):
    x = rs[0] * torch.sin(z_angle) * torch.cos(x_angle)
    y = rs[1] * torch.sin(z_angle) * torch.sin(x_angle)
    z = rs[2] * torch.cos(z_angle) 
    return torch.stack((x, y, z), dim=1)


rs = [0.9, 0.6, 0.3]
x_angle, z_angle = get_xz_angles(stacked)
ellipsoidal = get_ellipsoidal(x_angle, z_angle, rs)

faces = make_cube_faces(n)
vertices = to_vertices(ellipsoidal)
f = f'./cube_ellipsoid_{n}.stl'
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.export(f);

In [24]:
def get_xz_angles(bxyz):
    x, y, z = bxyz[:, 0, :, :], bxyz[:, 1, :, :], bxyz[:, 2, :, :]
    r = torch.sqrt(x**2 + y**2 + z**2)
    x_angle = torch.atan2(y, x)
    z_angle = torch.acos(z/r)
    return x_angle, z_angle

def get_ellipsoidal(x_angle, z_angle, rs):
    x = rs[0] * torch.sin(z_angle) * torch.cos(x_angle)
    y = rs[1] * torch.sin(z_angle) * torch.sin(x_angle)
    z = rs[2] * torch.cos(z_angle) 
    return torch.stack((x, y, z), dim=1)

class Ellipsoid(nn.Module):
    def __init__(self, n):
        super(Ellipsoid, self).__init__()        
        stacked = make_sides(n, stacked=True)         
        x_angle, z_angle = get_xz_angles(stacked)        
        self.register_buffer('x_angle', x_angle)
        self.register_buffer('z_angle', z_angle)
        self.register_buffer('faces', make_cube_faces(n).int())
        self.register_buffer('colors', torch.ones_like(vertices) * 0.5)


    def forward(self, rs):
        ellipsoidal = get_ellipsoidal(self.x_angle, self.z_angle, rs)        
        vert = to_vertices(ellipsoidal)
        return Mesh(vert, self.faces, self.colors)
    
n = 16
ellipsoid = Ellipsoid(n)
v, f, c = ellipsoid(rs)     
filename = f'./cube_ellipsoid_{n}.stl'
mesh = trimesh.Trimesh(vertices=v, faces=f)
mesh.export(filename);

In [21]:
v.shape, f.shape

AttributeError: 'str' object has no attribute 'shape'

In [23]:
f

'./cube_ellipsoid_16.stl'